### 패키지 불러오기

In [1]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance ## Feature Importance를 불러오기 위함
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 
from xgboost import plot_importance 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

### train/test 데이터셋 만들기 함수

In [7]:
#데이터셋 자동화
test = mean_test = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/test_mean_jo_sum.csv',encoding='utf-8')
#이건 USER_ID 붙이기용 테스트셋
mean_train = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/train_mean_jo_sum_final.csv', encoding='utf-8')
mean_test = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/test_mean_jo_sum.csv',encoding='utf-8')
non_train = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/train_nonanswer_jo_sum_final.csv', encoding='utf-8')
non_test = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/test_nonanswer_jo_sum.csv',encoding='utf-8')
mca_train = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/train_mca_jo_sum_final.csv', encoding='utf-8')
mca_test = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/test_mca_jo_sum.csv',encoding='utf-8')
rf_train = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/train_rf_jo_sum_final.csv', encoding='utf-8')
rf_test = pd.read_csv('C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/test_rf_jo_sum.csv',encoding='utf-8')
#train/test 데이터셋 만들어주는 함수
def train_test(train,test):
    y_train = train['Party']
    X_train = train.drop(['Party', 'USER_ID'],axis = 1)
    #X_train = pd.get_dummies(X,columns=['Gender', 'HouseholdStatus', 'age', 'marriage', 'kids'])
    #test = pd.get_dummies(test,columns=['Gender', 'HouseholdStatus', 'age', 'marriage', 'kids'])
    X_test = test.drop(['USER_ID'],axis=1)
    #트리 계열은 모든 변수 원핫인코딩 해야된대서.. 그렇게 해보자!
    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)
    y_train = pd.get_dummies(y_train, drop_first=True)
    return X_train, X_test, y_train
# 제출 형식 자동 맞춤 함수
def submission_form(y_pred,test_for_ID):
    target_data = pd.DataFrame(y_pred)
    target_data.columns = ['Predictions']
    target_data.loc[target_data["Predictions"] == 0,"Predictions"] = "Democrat"
    target_data.loc[target_data["Predictions"] == 1,"Predictions"] = "Republican"
    submission = pd.concat([test['USER_ID'],target_data],axis=1)
    return submission
# CSV파일로 변환해주는 함수
def to_csv(y_pred,test,name):
    submission = submission_form(y_pred,test)
    submission.to_csv("C:/Users/김서윤/Documents/통계 학회 P-SAT/주제 분석/주제분석 2주차 취합 모음/modeling data/submission/submission(%s).csv"%(name),header=True,index=False)

### XGBoost+Kfold+GridSearchCV
##### https://blog.naver.com/PostView.nhn?blogId=gustn3964&logNo=221431933811&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView

In [3]:
#여기서 어떤 데이터셋 사용할지 선택: non-answer
X_train, X_test, y_train = train_test(non_train,non_test)

In [4]:
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier

In [6]:
# 모델 설정
model= XGBClassifier()
# K-fold
cv=KFold(n_splits=5, random_state=1)
# 초매개변수 튜닝(CV, Grid Search)
param_grid={
    'max_depth':[3,4,5],
    'learning_rate':[1,0.5,0.1,0.01,0.05],
    'gamma':[0,0.25,1.0],
    'reg_lambda':[0,1.0,10.0,20,100],
    'scale_pos_weight':[1,3,5]}

In [ ]:
#돌아가는데 개오래걸리니 다시 돌릴거 아니면 건들지 말 것
model = XGBClassifier()
clf = GridSearchCV(model,param_grid=param_grid,cv=cv)
grid_search = clf.fit(X_train,y_train)

In [ ]:
# 최적 파라미터 구함
print("final params",clf.best_params_)

In [ ]:
#### 최적 파라미터 적용한 모델 #### : best_model
best_model = grid_search.best_estimator_
# Y값 예측
y_pred = best_model.predict(X_test)

In [ ]:
to_csv(y_pred,test,name="nonanswer+XG+Kfold+Grid")

#### shapley value 구하기

In [10]:
import shap

In [ ]:
explainer = shap.TreeExplainer(best_model)

In [ ]:
shap_values = explainer.shap_values(X_test)
shap.initjs()

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[1,:], X_test.iloc[1,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test) 

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type = "bar")

In [ ]:
# 변수 간의 shap value 파악
shap.dependence_plot("Income", shap_values, X_test)